# DIPC_PMS   
Solves PMS trigger on DIPC files

In [1]:
import pandas as pd
import math
import numpy as np
from glob import glob
pd.set_option("display.max_columns", None) #show all columns

In [2]:
# create list of all DIPC csv files
DIPC_files = sorted(glob('DIPCER_*'))
DIPC_files[0:4]

['DIPCER_202311100100.csv',
 'DIPCER_202311100200.csv',
 'DIPCER_202311100300.csv',
 'DIPCER_202311100400.csv']

In [3]:
# load and concatinate all DIPC csv files into one dataframe
DIPC = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)
          for file in DIPC_files),ignore_index = True)
print(DIPC.columns)
print(DIPC.shape)
DIPC.head()

<ipython-input-3-b4cb4a89f192>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  DIPC = pd.concat((pd.read_csv(file, skipfooter=1).assign(filename = file)


Index(['TIME_INTERVAL', 'REGION_NAME', 'RESOURCE_NAME', 'PRICING_FLAG', 'LMP',
       'SCHED_MW', 'LMP_SMP', 'LMP_LOSS', 'LMP_CONGESTION', 'Unnamed: 9',
       'filename'],
      dtype='object')
(283392, 11)


,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename
0,11/10/2023 12:05:00 AM,LUZON,01ACNPC_G01,OK,4720.9282,1.0000,4641.1013,79.8269,0.0,NaN,DIPCER_202311100100.csv
1,11/10/2023 12:05:00 AM,LUZON,01AEC_L01,OK,4713.9666,-46.6268,4641.1013,72.8653,0.0,NaN,DIPCER_202311100100.csv
2,11/10/2023 12:05:00 AM,LUZON,01AMBUK_SS,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv
3,11/10/2023 12:05:00 AM,LUZON,01AMBUK_T1L1,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv
4,11/10/2023 12:05:00 AM,LUZON,01AMBUK_U01,OK,4727.8899,5.0000,4641.1013,86.7886,0.0,NaN,DIPCER_202311100100.csv


In [4]:
#Converting schedule to positive values
DIPC['SCHED_MW'] = abs(DIPC['SCHED_MW'])
#DIPC2 uses only resources from Visayas and Luzon for comparison
DIPC2 = DIPC[DIPC['REGION_NAME']!= 'MINDANAO'].copy()
#DIPC2 = DIPC[DIPC['REGION_NAME']== 'VISAYAS'].copy()
#DIPC2 = DIPC.copy()
DIPC2.tail()

,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename
283137,11/11/2023,VISAYAS,08STBPB1_U02,OK,5503.1769,0.0,4511.1705,992.0064,0.0,NaN,DIPCER_202311110000.csv
283138,11/11/2023,VISAYAS,08STBPB1_U03,OK,5503.1769,0.0,4511.1705,992.0064,0.0,NaN,DIPCER_202311110000.csv
283139,11/11/2023,VISAYAS,08STBPB1_U04,OK,5503.1769,0.0,4511.1705,992.0064,0.0,NaN,DIPCER_202311110000.csv
283140,11/11/2023,VISAYAS,08SUWECO_G01,OK,5502.7258,3.6,4511.1705,991.5553,0.0,NaN,DIPCER_202311110000.csv
283141,11/11/2023,VISAYAS,08TIMBA_G01,OK,5780.6139,12.0,4511.1705,1269.4434,0.0,NaN,DIPCER_202311110000.csv


In [5]:
#Solving for weighted price by multiplying price by schedule
DIPC2['SKEDXLMP'] = DIPC2['SCHED_MW']*DIPC2['LMP']


In [6]:
#summing all rows by time interval
DIPC4 = DIPC2.groupby(['TIME_INTERVAL'],sort=False).sum().copy().reset_index()
DIPC4.head()

,TIME_INTERVAL,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,SKEDXLMP
0,11/10/2023 12:05:00 AM,3.554833e+06,22294.5357,3.406568e+06,148264.6212,0.0,0.0,1.038730e+08
1,11/10/2023 12:10:00 AM,3.486528e+06,22238.3758,3.335226e+06,151302.4658,0.0,0.0,1.015492e+08
2,11/10/2023 12:15:00 AM,3.476419e+06,22168.1818,3.334887e+06,141531.4279,0.0,0.0,1.010856e+08
3,11/10/2023 12:20:00 AM,3.470129e+06,22080.7963,3.333534e+06,136595.4027,0.0,0.0,1.005887e+08
4,11/10/2023 12:25:00 AM,3.458519e+06,21983.0419,3.331506e+06,127013.4219,0.0,0.0,9.992080e+07


In [7]:
#NWAP is then solved as SKEDxLMP divided by SCHED_MW since all rows are now the sum per interval
DIPC4['NWAP']= DIPC4['SKEDXLMP']/DIPC4['SCHED_MW']
DIPC4.head()

,TIME_INTERVAL,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,SKEDXLMP,NWAP
0,11/10/2023 12:05:00 AM,3.554833e+06,22294.5357,3.406568e+06,148264.6212,0.0,0.0,1.038730e+08,4659.124843
1,11/10/2023 12:10:00 AM,3.486528e+06,22238.3758,3.335226e+06,151302.4658,0.0,0.0,1.015492e+08,4566.394158
2,11/10/2023 12:15:00 AM,3.476419e+06,22168.1818,3.334887e+06,141531.4279,0.0,0.0,1.010856e+08,4559.942347
3,11/10/2023 12:20:00 AM,3.470129e+06,22080.7963,3.333534e+06,136595.4027,0.0,0.0,1.005887e+08,4555.482867
4,11/10/2023 12:25:00 AM,3.458519e+06,21983.0419,3.331506e+06,127013.4219,0.0,0.0,9.992080e+07,4545.358219


In [8]:
#Preparing dataframe for merging
DIPC5 = DIPC4[['TIME_INTERVAL','SKEDXLMP','NWAP']].copy()
DIPC5 = DIPC5.rename(columns={'SKEDXLMP': 'SKEDXLMP_SUM'})

DIPC5.head()

,TIME_INTERVAL,SKEDXLMP_SUM,NWAP
0,11/10/2023 12:05:00 AM,1.038730e+08,4659.124843
1,11/10/2023 12:10:00 AM,1.015492e+08,4566.394158
2,11/10/2023 12:15:00 AM,1.010856e+08,4559.942347
3,11/10/2023 12:20:00 AM,1.005887e+08,4555.482867
4,11/10/2023 12:25:00 AM,9.992080e+07,4545.358219


In [9]:
DIPC6 = DIPC2.merge(DIPC5, on='TIME_INTERVAL', how='left').copy()
DIPC6.head()

,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename,SKEDXLMP,SKEDXLMP_SUM,NWAP
0,11/10/2023 12:05:00 AM,LUZON,01ACNPC_G01,OK,4720.9282,1.0000,4641.1013,79.8269,0.0,NaN,DIPCER_202311100100.csv,4720.928200,1.038730e+08,4659.124843
1,11/10/2023 12:05:00 AM,LUZON,01AEC_L01,OK,4713.9666,46.6268,4641.1013,72.8653,0.0,NaN,DIPCER_202311100100.csv,219797.177865,1.038730e+08,4659.124843
2,11/10/2023 12:05:00 AM,LUZON,01AMBUK_SS,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv,0.000000,1.038730e+08,4659.124843
3,11/10/2023 12:05:00 AM,LUZON,01AMBUK_T1L1,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv,0.000000,1.038730e+08,4659.124843
4,11/10/2023 12:05:00 AM,LUZON,01AMBUK_U01,OK,4727.8899,5.0000,4641.1013,86.7886,0.0,NaN,DIPCER_202311100100.csv,23639.449500,1.038730e+08,4659.124843


In [10]:
#solving for value 'X' to make price trigger solving easier which is simply the product of schedule and (price minus nwap) squared
DIPC6['X'] = DIPC6['SCHED_MW']*((DIPC6['LMP']-DIPC6['NWAP'])*(DIPC6['LMP']-DIPC6['NWAP']))
DIPC6.head()

,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,PRICING_FLAG,LMP,SCHED_MW,LMP_SMP,LMP_LOSS,LMP_CONGESTION,Unnamed: 9,filename,SKEDXLMP,SKEDXLMP_SUM,NWAP,X
0,11/10/2023 12:05:00 AM,LUZON,01ACNPC_G01,OK,4720.9282,1.0000,4641.1013,79.8269,0.0,NaN,DIPCER_202311100100.csv,4720.928200,1.038730e+08,4659.124843,3819.654942
1,11/10/2023 12:05:00 AM,LUZON,01AEC_L01,OK,4713.9666,46.6268,4641.1013,72.8653,0.0,NaN,DIPCER_202311100100.csv,219797.177865,1.038730e+08,4659.124843,140235.617672
2,11/10/2023 12:05:00 AM,LUZON,01AMBUK_SS,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv,0.000000,1.038730e+08,4659.124843,0.000000
3,11/10/2023 12:05:00 AM,LUZON,01AMBUK_T1L1,OK,4728.3540,0.0000,4641.1013,87.2527,0.0,NaN,DIPCER_202311100100.csv,0.000000,1.038730e+08,4659.124843,0.000000
4,11/10/2023 12:05:00 AM,LUZON,01AMBUK_U01,OK,4727.8899,5.0000,4641.1013,86.7886,0.0,NaN,DIPCER_202311100100.csv,23639.449500,1.038730e+08,4659.124843,23643.165350


In [11]:
#summing all rows by time interval
DIPC7 = DIPC6.groupby(['TIME_INTERVAL'],sort=False).sum().copy().reset_index()
DIPC7 = DIPC7[['TIME_INTERVAL','SCHED_MW','X']].copy()
DIPC7.head()

,TIME_INTERVAL,SCHED_MW,X
0,11/10/2023 12:05:00 AM,22294.5357,2.262835e+09
1,11/10/2023 12:10:00 AM,22238.3758,2.157494e+09
2,11/10/2023 12:15:00 AM,22168.1818,1.874525e+09
3,11/10/2023 12:20:00 AM,22080.7963,1.747190e+09
4,11/10/2023 12:25:00 AM,21983.0419,1.544786e+09


In [12]:
#adding the NWAP value back because the new 
DIPC8 = DIPC7.merge(DIPC5, on='TIME_INTERVAL', how='left').copy()
DIPC8.head()

,TIME_INTERVAL,SCHED_MW,X,SKEDXLMP_SUM,NWAP
0,11/10/2023 12:05:00 AM,22294.5357,2.262835e+09,1.038730e+08,4659.124843
1,11/10/2023 12:10:00 AM,22238.3758,2.157494e+09,1.015492e+08,4566.394158
2,11/10/2023 12:15:00 AM,22168.1818,1.874525e+09,1.010856e+08,4559.942347
3,11/10/2023 12:20:00 AM,22080.7963,1.747190e+09,1.005887e+08,4555.482867
4,11/10/2023 12:25:00 AM,21983.0419,1.544786e+09,9.992080e+07,4545.358219


In [13]:
#solving for price trigger factor
DIPC8['PRICE_TRIGGER_FACTOR'] = abs((np.sqrt(abs(DIPC8['X']/DIPC8['SCHED_MW'])))/DIPC8['NWAP'])
DIPC8.head()

,TIME_INTERVAL,SCHED_MW,X,SKEDXLMP_SUM,NWAP,PRICE_TRIGGER_FACTOR
0,11/10/2023 12:05:00 AM,22294.5357,2.262835e+09,1.038730e+08,4659.124843,0.068379
1,11/10/2023 12:10:00 AM,22238.3758,2.157494e+09,1.015492e+08,4566.394158,0.068210
2,11/10/2023 12:15:00 AM,22168.1818,1.874525e+09,1.010856e+08,4559.942347,0.063771
3,11/10/2023 12:20:00 AM,22080.7963,1.747190e+09,1.005887e+08,4555.482867,0.061749
4,11/10/2023 12:25:00 AM,21983.0419,1.544786e+09,9.992080e+07,4545.358219,0.058321


In [14]:
DIPC8.to_csv('PSM.csv')